In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow import keras 
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from skimage.transform import resize
import tensorflow

sns.set(style='white', context='notebook', palette='deep')

tensorflow.config.list_physical_devices("GPU")

ModuleNotFoundError: No module named 'seaborn'

In [3]:
X = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")
y = X.pop("label")
y = to_categorical(y, num_classes = 10)
# Normalize the data
X = X / 255.0
X_test = X_test / 255.0
X = X.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)


##klooien met de afmetingen!
#X = np.pad(X, ((0,0), (2,2), (2,2), (0,0)), mode='constant')
#add rgb channel
#X = np.squeeze(X, axis=-1)
#X = stacked_img = np.stack((X,)*3, axis=-1)


In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.1, random_state=1337)



In [5]:
X_train.shape

(37800, 28, 28, 1)

In [2]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"])

NameError: name 'Sequential' is not defined

In [7]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [1]:
history = model.fit(datagen.flow(X_train,y_train, batch_size=86),
                              epochs = 30, validation_data = (X_valid,y_valid),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // 86
                             # , callbacks=[learning_rate_reduction]
                             )

NameError: name 'model' is not defined

In [ ]:
# Look at confusion matrix 

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_valid)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_valid,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
# predict results
results = model.predict(X_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)

In [44]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.models import Model
from skimage.transform import resize
from skimage import color
import efficientnet.tfkeras as efn
import cv2

train = pd.read_csv("data/train.csv")

Y_train = train['label'].values.astype('float32')
Y_train = tensorflow.keras.utils.to_categorical(Y_train, 10)
X_train = train.drop(labels=['label'], axis=1)
X_train = X_train.astype('float32')
#normalize
X_train = X_train / 255
#make into image matrixes
X_train = X_train.values.reshape(42000,28,28,1)
#blow up
X_train = np.pad(X_train, ((0,0), (2,2), (2,2), (0,0)), mode='constant')
#add rgb channel
X_train = np.squeeze(X_train, axis=-1)
X_train = stacked_img = np.stack((X_train,)*3, axis=-1)


In [52]:
enet = efn.EfficientNetB0(
input_shape=(32, 32, 3),
weights='imagenet',
include_top=False,
)          

enet.trainable = False

model =Sequential([
    enet,
    layers.Flatten(),
    Dense(1024, activation="relu"),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
    

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(
    datagen.flow(X_train,Y_train, batch_size=86),
    epochs=2,
    steps_per_epoch=X_train.shape[0] // 86,
    verbose = 1
)

16809984/16804768 [==============================] - 2s 0us/step
Epoch 1/2
488/488 [==============================] - 14s 29ms/step - loss: 1.1288 - accuracy: 0.6008
Epoch 2/2
488/488 [==============================] - 14s 29ms/step - loss: 0.8511 - accuracy: 0.7070
